# End to end example on certifying local explanations using `Ecertify`

In this notebook, we demonstrate how to _certify_ a local explanation for a prediction of a classification 
model. Here we choose the popular tabular dataset [FICO HELOC](https://github.com/Trusted-AI/AIX360/blob/master/examples/tutorials/HELOC.ipynb), and use local explainers such as [LIME](https://github.com/marcotcr/lime) and [SHAP](https://github.com/shap/shap) for certification. We also comment
on how the explainers can be compared on the basis of their (found) certification widths ($w$) at the end.
The cells below describe steps needed to perform certification in detail:

1. obtaining a trained model on the dataset, here we use `GradientBoostingClassifier` from the sklearn library
2. selecting an instance of interest and computing its explanation
3. defining the quality criterion (here we use `1 - mean absolute error`, i.e., the fidelity as mentioned in the paper) to assess the degree to which the computed explanation is _applicable_ to other instances
4. decide the fidelity threshold $\theta$, this is another user configurable option
5. certify the explanation, i.e., find the largest hypercube around the original instance where the computed explanation has sufficiently high fidelity $\ge \theta$

In [1]:
# import sys; sys.path.append('../../aix360/')

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os, math, timeit, pickle
from datetime import datetime
import numpy as np; np.set_printoptions(suppress=True)
import pandas as pd


# sklearn utilities
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, r2_score


# explainers
import shap, lime
from lime.lime_tabular import LimeTabularExplainer

# our code
from aix360.algorithms.ecertify.utils import load_fico_dataset, compute_lime_explainer, compute_shap_explainer
# from algorithms.ecertify.utils import load_fico_dataset, compute_lime_explainer, compute_shap_explainer

## Load dataset and prepare the model

In [3]:
random_state=1234

# load FICO data and split into train-test sets
df, y = load_fico_dataset(path='./data/heloc-clean-full.csv')
x_train, x_test, y_train, y_test = train_test_split(df, y, train_size=0.7, random_state=random_state)
is_regression = False # this is a classification dataset

# standardization
EXtr, StdXtr = x_train.mean(0), x_train.std(0)
x_train = (x_train - EXtr) / StdXtr
x_test = (x_test - EXtr) / StdXtr


# fit a gradient boosted classifier on this dataset
model = GradientBoostingClassifier(random_state=random_state)
model.fit(x_train, y_train)

# check accuracy
print(f"train accuracy: {np.around(accuracy_score(y_train, model.predict(x_train)), 4) * 100}%")
print(f"test accuracy: {np.around(accuracy_score(y_test, model.predict(x_test)), 4) * 100}%")

train accuracy: 85.03%
test accuracy: 82.55%


## Prepare the callables for querying the model and the explanation during certification
The next cell prepares few callables: `_bb()` and `_e()` to query the `model` and the explanation. Note that
we are assuming we have obtained a functional form of the computed explainer which can be _applied_ to another
instance. In case of LIME, the explanation is a linear function with weights/coefficients set as the feature
importance values in the explanation. For KernelSHAP, a similar linear function is used as well. We apply the
function on an instance and get the correct class' (the original instance's class for which the explanation 
was computed) probability.

Later when we have the `model` and the `expl_func` ready, we can wrap these functions(`_bb()` and `_e()`) with
the `partial` _functool_ to hide the second argument (e.g., `expl_func` in `_e()`) and only pass the first argument `x`.

In [7]:
from functools import partial

def _bb(x, model, label_x0=0, is_regression=False):
    """
        x: single 1d numpy array of shape (d, )
        label_x0: if classification, we need to take the correct class' probability
    """
    x = [x]

    if is_regression:
        return model.predict(x)[0]
    else:
        return model.predict_proba(x)[:, label_x0][0]


def _e(x, expl_func):
    """
        x: single 1d numpy array of shape (d, )
        expl_func: a callable/sklearn model with predict method
    """

    x = [x]
    return expl_func.predict(x)[0]

## Choose a random sample for finding explanation and certification
Here we choose one prototype that we also discussed in the paper.

In [8]:
sample_idx = 6863

x0 = x_train.iloc[sample_idx]
true_label = y_train.iloc[sample_idx]

# Get the output of the black-box classifier on x0
output = model.predict_proba([x0])[0]
label_x0 = np.argmax(output)
prob_x0 = output[label_x0]

print(f'data index={sample_idx}')
print(f"model class probs output: {np.around(output, 4)}, ground truth label: {true_label}")
print(f'predicted prob: {prob_x0:.4f}, predicted label: {label_x0}')

data index=6863
model class probs output: [0.8918 0.1082], ground truth label: 0.0
predicted prob: 0.8918, predicted label: 0


In [9]:
# view the instance x0
(x_train.iloc[sample_idx:sample_idx+1] * StdXtr + EXtr).T

,8165
ExternalRiskEstimate,49.0
MSinceOldestTradeOpen,200.0
MSinceMostRecentTradeOpen,2.0
AverageMInFile,71.0
NumSatisfactoryTrades,45.0
NumTrades60Ever2DerogPubRec,0.0
NumTrades90Ever2DerogPubRec,0.0
PercentTradesNeverDelq,98.0
MSinceMostRecentDelq,4.0
MaxDelq2PublicRecLast12M,4.0


In [11]:
# prepare the blackbox function for querying the model, partial magic
bb = partial(_bb, model=model, label_x0=label_x0, is_regression=False)

## Check fidelity of explanations on the point `x0` itself

Note that LIME explanations can have fidelity less than 1.0 on that instance, i.e., the linear/affine function 
approximated the model at $x_0$ need not pass through the model's predicted probability for $x_0$. But for 
KernelSHAP, the approximating linear function always passes through model's predicted probability (this is
also known as the efficiency criterion for shapley values, i.e., the sum of values should be equal to the 
total value).

In [12]:
print("test lime: ")
# test the function from utilities
func, expl = compute_lime_explainer(x_train, model, x0, num_features=len(x0))
e = partial(_e, expl_func=func)
f = lambda x: 1 - abs(bb(x) - e(x))  # fidelity function (specific to this explanation)

print(f"bb(x)={bb(x0.values):.4f}, e(x)={e(x0.values):.4f}")
print(f"fidelity at x0: f(x)={f(x0.values):.4f}")
print('-'*40)



print("test shap: ")

func, expl = compute_shap_explainer(x_train, model, x0)
e = partial(_e, expl_func=func)
f = lambda x: 1 - abs(bb(x) - e(x))  # fidelity function (specific to this explanation)

print(f"bb(x)={bb(x0.values):.4f}, e(x)={e(x0.values):.4f}")
print(f"fidelity at x0: f(x)={f(x0.values):.4f}")

print('-'*40)

test lime: 
bb(x)=0.8918, e(x)=0.7679
fidelity at x0: f(x)=0.8761
----------------------------------------
test shap: 
bb(x)=0.8918, e(x)=0.8918
fidelity at x0: f(x)=1.0000
----------------------------------------


# Certification
For this step, we would need callables $bb(.)$, $e(.)$ and $f(.)$ that we prepared earlier. Note that the 
algorithm only requires the user to define a suitable fidelity function $f(.)$ that subsumes the calls to the 
blackbox model, $bb(.)$, and the local explanation function, $e(.)$.

In [16]:
from aix360.algorithms.ecertify.ecertify import CertifyExplanation

First we certify for the LIME explanation, we are computing again since in the last cell the variables were
overwritten by the SHAP explanation.

In [17]:
func, expl = compute_lime_explainer(x_train, model, x0, num_features=len(x0))
e = partial(_e, expl_func=func)
f = lambda x: 1 - abs(bb(x) - e(x))  # fidelity function (specific to this explanation)

## Initialize the certifier object

In [22]:
# Inputs to Certify()
theta = 0.75   # user desired fidelity threshold                                     
lb = 0; ub = 1 # init hypercube of size 1
Q = 10000      # query budget related arguments
Z = 10         # number of halving/doubling iterations during certification (so total queries expensed = Z*Q)
sigma0 = 0.1   # sigma for gaussians used in unifI and adaptI strategies
NUMRUNS = 10   # consider running for more iterations here for reduced error

certifier = CertifyExplanation(theta=theta, Q=Q, Z=Z, lb=lb, ub=ub, sigma0=sigma0, numruns=NUMRUNS)

In [23]:
x = x0.values
s = 3

w = certifier.certify_instance(instance=x, quality_criterion=f, strategy=s, silent=False)

Time per run: 6.844 s
Found w: 0.4298 ± 0.047384


## Certification of SHAP
Similarly we could also certify the KernelSHAP explanation for the same instance, just using a properly defined quality criterion that takes the SHAP `expl_func` object.

In [24]:
# get the KernelSHAP explanation
func, expl = compute_shap_explainer(x_train, model, x0)
e = partial(_e, expl_func=func)
f = lambda x: 1 - abs(bb(x) - e(x))  # fidelity function (specific to this explanation)

In [25]:
x = x0.values
s = 3

w = certifier.certify_instance(instance=x, quality_criterion=f, strategy=s, silent=False)

Time per run: 4.467 s
Found w: 0.0272 ± 0.002681


## Observation
Note that for this instance, the LIME explanation has a larger certifier width ($\approx 0.3-0.4$) than the 
KernelSHAP explanation ($\approx 0.02-0.04$), implying the linear explanation obtained from LIME is applicable 
to a relatively large neighbourhood than the corresponding KernelSHAP explanation.